In [6]:
import os
import glob

import numpy as np
import xarray as xr

from frites.utils import savgol_filter

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio

pio.templates.default = 'plotly_white'

# %matplotlib notebook
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


---

# **Load epochs**

In [20]:
################################################################################
# file settings
reference = 'sample_stim'  # {'sample_stim', 'sample_resp'}
freq = "f50f150"  # {"f50f150", "f8f24"}
smoothing = "sm0"

# folders location
analysis = '/hpc/brainets/data/db_ebrains/analysis'
################################################################################

# build the path where the epochs are saved
path_epochs = os.path.join(analysis, 'epochs', reference, f"{freq}-{smoothing}", 'data')
assert os.path.isdir(path_epochs)
files = glob.glob(os.path.join(path_epochs, "*.nc"))

# loop over the files
data = []
for n_f, f in enumerate(files):
    print(f"Loading subject #{n_f}", end="\r")
    _data = xr.load_dataarray(f)
    _data = _data.sel(times=slice(-2., 3.))
    _data = _data.drop('contacts')
    _data = _data.groupby('trials').mean('trials')
    _data = _data.groupby('roi').mean('roi')
    _data = savgol_filter(_data, 1., axis='times', verbose=False)
    data += [_data]
data = xr.concat(data, 'roi').groupby('roi').mean('roi')

In [21]:
roi_plt = [
    'Insula', 'OFCv', 'OFCvl',
    'PMdm', 'PMdl', 'Mdl',
    'Sdl', 'SPC', 'SPCm',
    'STCc',
    'VCcm', 'VCl'
]

# append left and right hemispheres
roi_lr = []
for r in roi_plt:
    roi_lr.append('L_' + r)
    roi_lr.append('R_' + r)
data_lr = data.sel(roi=roi_lr)

# dataframe conversion
df = data_lr.to_dataframe(freq).reset_index()

In [22]:
fig = px.line(
    df, x='times', y=freq, color='trials', facet_col='roi',
    facet_col_wrap=6
)
fig.add_vline(0.)
fig.update_yaxes(matches=None)
fig.update_yaxes(showticklabels=False)
fig